# 1. Install packages

In [ ]:
!git clone https://github.com/openai/improved-diffusion.git
%cd improved-diffusion
!pip install -e .

In [ ]:
!pip install mpi4py

# 2. Setting for training

In [3]:
import os
os.environ["OPENAI_LOGDIR"] = "/kaggle/working/checkpoints"

In [4]:
MODEL_FLAGS="--image_size 256 --num_channels 128 --num_res_blocks 2 --num_heads 1 --learn_sigma True --use_scale_shift_norm False --attention_resolutions 16"
DIFFUSION_FLAGS="--diffusion_steps 1000 --noise_schedule linear --rescale_learned_sigmas False --rescale_timesteps False --use_scale_shift_norm False"
TRAIN_FLAGS="--lr 2e-5 --batch_size 4 --lr_anneal_steps 132000 --log_interval 3000 --save_interval 36000"

## 2.1 Training

In [ ]:
!python scripts/image_train.py \
    --data_dir /kaggle/input/train-dataset/train-dataset \
    $MODEL_FLAGS $DIFFUSION_FLAGS $TRAIN_FLAGS

## 2.2 Sampling

In [ ]:
!python scripts/image_sample.py --model_path /kaggle/input/pretrained-132k/pytorch/default/1/132k/ema_0.9999_132000.pt --batch_size 4 --num_samples 152 $MODEL_FLAGS $DIFFUSION_FLAGS

In [6]:
import numpy as np
from PIL import Image
import os

data = np.load("/kaggle/working/checkpoints/samples_12x256x256x3.npz")
images = data["arr_0"]

os.makedirs("/kaggle/working/samples_png", exist_ok=True)
for i, img in enumerate(images):
    Image.fromarray(img).save(f"/kaggle/working/samples_png/{i:05d}.png")


# 3. Evaluation

In [ ]:
!pip install pytorch-fid

In [ ]:
!python -m pytorch_fid /kaggle/input/dev-dataset/dev-dataset /kaggle/working/samples_png --device cuda:0